<a href="https://colab.research.google.com/github/Sirabhop/MFP-Volunteer/blob/main/20220619_DRR_and_DoR_Spending_Budget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

In [2]:
os.chdir('/content/drive/MyDrive/My Project/MFP_Volunteer/20220619_DRRandDoR')

###Prelim function

In [3]:
def track_status(df, col_list):
  print('Row:', "{:,}".format(len(df.index)), 'and', 'Columns:', "{:,}".format(len(df.columns)))
  for i in range(len(col_list)):
    print(col_list[i]+':', "{:,}".format(df[col_list[i]].sum()))

In [4]:
def extract_from_specific_word(df, col_to_extract, word, range_):
  to_extract = df[col_to_extract].values
  extracted = []
  for i in range(len(to_extract)):
    km_index = str(to_extract[i]).find(word)
    if km_index != -1:
      km = to_extract[i][km_index: km_index + range_]
      km = ''.join(re.findall("[0-9]", km))
      extracted.append(str(km))
    else:
      extracted.append(np.NaN)
  return extracted

In [5]:
def find_start_end(df, col_n_start, col_n_end):
  start = []
  end = []
  years = df.columns[col_n_start:col_n_end]
  r_years = list(reversed(years))
  nrow = len(df.index)
  ncol = len(years)
  for row_number in range(nrow):
    for col_number in range(ncol):
      if df.loc[row_number, years[col_number]] == 1:
        s = years[col_number]
        break
    for col_number in range(ncol):
      if df.loc[row_number, r_years[col_number]] == 1:
        e = r_years[col_number]
        break
    start.append(s)
    end.append(e)
    print(row_number, s, e)
  return start, end

###Import data

In [6]:
df = pd.read_excel('/content/drive/MyDrive/My Project/MFP_Volunteer/งบประมาณประเทศไทย 2566 (ฉบับร่างพ.ร.บ.) - PDF to CSV.xlsx', sheet_name = 'RELEASE_22-05-27')

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [7]:
df_MoT = df[df['MINISTRY'] == 'กระทรวงคมนาคม']
df_MoT.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6337 entries, 34001 to 40339
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   REF_DOC           6337 non-null   object 
 1   REF_PAGE_NO       6337 non-null   float64
 2   MINISTRY          6337 non-null   object 
 3   BUDGETARY_UNIT    6337 non-null   object 
 4   BUDGET_PLAN       6337 non-null   object 
 5   CROSS_FUNC?       6337 non-null   bool   
 6   PROJECT           3716 non-null   object 
 7   OUTPUT            2557 non-null   object 
 8   CATEGORY_LV1      6337 non-null   object 
 9   CATEGORY_LV2      6191 non-null   object 
 10  CATEGORY_LV3      5764 non-null   object 
 11  CATEGORY_LV4      5228 non-null   object 
 12  CATEGORY_LV5      337 non-null    object 
 13  CATEGORY_LV6      0 non-null      float64
 14  ITEM_DESCRIPTION  6337 non-null   object 
 15  FISCAL_YEAR       6337 non-null   float64
 16  OBLIGED?          6337 non-null   boo

In [8]:
track_status(df_MoT, ['AMOUNT'])

Row: 6,337 and Columns: 18
AMOUNT: 409,051,732,300.0


In [9]:
df_MoT_RR = df_MoT[df_MoT['BUDGETARY_UNIT'].isin(['กรมทางหลวง'])]
track_status(df_MoT_RR, ['AMOUNT'])

Row: 3,604 and Columns: 18
AMOUNT: 308,323,668,800.0


In [30]:
df_focused = df_MoT_RR[df_MoT_RR['BUDGET_PLAN'] == 'แผนงานบูรณาการพัฒนาด้านคมนาคมและระบบโลจิสติกส์']
track_status(df_focused, ['AMOUNT'])

Row: 2,314 and Columns: 18
AMOUNT: 260,514,448,700.0


####1. Extract main item

In [31]:
df_focused[['MAIN', 'TO_DROP', 'MAIN_2']] = df_focused.ITEM_DESCRIPTION.str.partition()
df_focused.reset_index(drop = True, inplace = True)
df_focused.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2314 entries, 0 to 2313
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   REF_DOC           2314 non-null   object 
 1   REF_PAGE_NO       2314 non-null   float64
 2   MINISTRY          2314 non-null   object 
 3   BUDGETARY_UNIT    2314 non-null   object 
 4   BUDGET_PLAN       2314 non-null   object 
 5   CROSS_FUNC?       2314 non-null   bool   
 6   PROJECT           2314 non-null   object 
 7   OUTPUT            0 non-null      object 
 8   CATEGORY_LV1      2314 non-null   object 
 9   CATEGORY_LV2      2228 non-null   object 
 10  CATEGORY_LV3      2228 non-null   object 
 11  CATEGORY_LV4      2126 non-null   object 
 12  CATEGORY_LV5      298 non-null    object 
 13  CATEGORY_LV6      0 non-null      float64
 14  ITEM_DESCRIPTION  2314 non-null   object 
 15  FISCAL_YEAR       2314 non-null   float64
 16  OBLIGED?          2314 non-null   bool   


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
df_focused.drop(columns = 'TO_DROP', axis = 1, inplace = True)

In [13]:
track_status(df_focused, ['AMOUNT'])

Row: 2,314 and Columns: 20
AMOUNT: 260,514,448,700.0


####2. Extract distance

In [14]:
to_extract = df_focused['MAIN_2'].values
extracted = []
unit = []
for i in range(len(to_extract)):
  km_index = str(to_extract[i]).find('กม.')
  m_index = str(to_extract[i]).find(' ม.')
  if km_index != -1:
    km = to_extract[i][km_index - 8: km_index]
    km = ''.join(re.findall("[0-9,.]", km))
    extracted.append(km)
    unit.append('km')
  elif m_index != -1:
    m = to_extract[i][m_index - 8: m_index+1]
    m = ''.join(re.findall("[0-9,.]", m))
    extracted.append(m)
    unit.append('m')
  else:
    extracted.append(np.NaN)
    unit.append(np.NaN)

In [ ]:
df_focused['DISTANCE'] = extracted
df_focused['UNIT'] = unit

In [ ]:
df_focused['DISTANCE'] = df_focused['DISTANCE'].str.replace(',', '', regex = True)
df_focused.loc[df_focused['UNIT'] == 'm', 'DISTANCE'] = df_focused.loc[df_focused['UNIT'] == 'm', 'DISTANCE'].astype(float)/1000

In [ ]:
df_focused['DISTANCE'] = pd.to_numeric(df_focused['DISTANCE'], errors = 'coerce')

In [18]:
track_status(df_focused, ['AMOUNT', 'DISTANCE'])

Row: 2,314 and Columns: 22
AMOUNT: 260,514,448,700.0
DISTANCE: 12,779.386


####3. Extract province

In [ ]:
df_focused.insert(18, 'PROVINCE', np.NaN)
#Import province
province = pd.read_excel('/content/drive/MyDrive/My Project/MFP_Volunteer/25640531_sc002_.xlsx', usecols = ['ProvinceNameThai', 'RegionName'])
province.rename(columns = {'ProvinceNameThai': 'PROVINCE'}, inplace = True)
#Type 1: full province
province_type1_list = province.PROVINCE.to_list()
#Type 2: in case the item_description doesn't contain จังหวัด
province_type2_list = []
for i in range(len(province_type1_list)):
  province_type2_list.append(province_type1_list[i].replace('จังหวัด', ''))
for i in range(len(province_type2_list)):
  df_focused.loc[df_focused.ITEM_DESCRIPTION.str.contains(province_type2_list[i]), 'PROVINCE'] = province_type1_list[i]

In [20]:
track_status(df_focused, ['AMOUNT', 'DISTANCE'])

Row: 2,314 and Columns: 23
AMOUNT: 260,514,448,700.0
DISTANCE: 12,779.386


###Answer the question

####1. Obliged budget start from 2023

In [ ]:
#Get dummies 10 columns added
df_focused[['2016','2017','2018','2019','2020','2021','2022','2023','2024','2025']] = pd.get_dummies(df_focused['FISCAL_YEAR'])

In [37]:
track_status(df_focused, ['AMOUNT'])

Row: 2,314 and Columns: 32
AMOUNT: 260,514,448,700.0


In [66]:
df_distinct = df_focused.groupby(['ITEM_DESCRIPTION']).agg({'2016':'sum', '2017':'sum', '2018':'sum', '2019':'sum', '2020':'sum', 
                                                                    '2021':'sum', '2022':'sum', '2023':'sum', '2024':'sum', '2025':'sum', 
                                                                    'AMOUNT':'sum'})

In [67]:
track_status(df_distinct, ['AMOUNT'])

Row: 1,170 and Columns: 11
AMOUNT: 260,514,448,700.0


4. Extract road code

In [69]:
df_distinct.reset_index(inplace = True)

In [70]:
df_distinct['ROAD'] = extract_from_specific_word(df_distinct, 'ITEM_DESCRIPTION', 'ทางหลวงหมายเลข', 20)

In [71]:
track_status(df_distinct, ['AMOUNT'])

Row: 1,170 and Columns: 13
AMOUNT: 260,514,448,700.0


In [ ]:
start, end = find_start_end(df_distinct, 2, 12)

In [73]:
df_distinct['START'] = start
df_distinct['END'] = end

In [74]:
track_status(df_distinct, ['AMOUNT'])

Row: 1,170 and Columns: 15
AMOUNT: 260,514,448,700.0


In [75]:
df_distinct = pd.merge(df_distinct, df_focused[['ITEM_DESCRIPTION', 'BUDGET_PLAN', 
                                'PROJECT', 'OBLIGED?', 'CATEGORY_LV1', 'CATEGORY_LV2', 
                                'CATEGORY_LV3', 'CATEGORY_LV4', 'PROVINCE']],
                                how = 'inner', on = 'ITEM_DESCRIPTION')

In [76]:
track_status(df_distinct, ['AMOUNT'])

Row: 2,314 and Columns: 23
AMOUNT: 1,186,603,530,000.0


In [77]:
df_distinct.drop_duplicates(inplace = True)

In [78]:
track_status(df_distinct, ['AMOUNT'])

Row: 1,170 and Columns: 23
AMOUNT: 260,514,448,700.0


In [79]:
df_distinct['DURATION'] = pd.to_numeric(df_distinct['END']) - pd.to_numeric(df_distinct['START']) + 1

In [80]:
track_status(df_distinct, ['AMOUNT'])

Row: 1,170 and Columns: 24
AMOUNT: 260,514,448,700.0


In [81]:
df_distinct['ROAD_LV'] = df_distinct['ROAD'].str.len()

In [82]:
track_status(df_distinct, ['AMOUNT'])

Row: 1,170 and Columns: 25
AMOUNT: 260,514,448,700.0


####4. Export

In [ ]:
df_MoT.to_excel('20220619_MoT.xlsx', index=False)

In [ ]:
df_MoT_RR.to_excel('20220619_MoT_province.xlsx', index=False)

In [83]:
df_distinct.to_excel('20220623_MoT_final_DoR_2.xlsx', index = False)

In [ ]:
df_distinct.to_excel('20220623_MoT_DoR_2023.xlsx', index = False)